In [26]:
import time
import cobra
import cobrakbase
start = time.time()

In [27]:
api = cobrakbase.KBaseAPI('DGXHUU34PUCT2V5JGUYA23P6TMQJM3FK')
ws = 'filipeliu:narrative_1592958114519'

In [28]:
kbase_model = api.get_object('GCF_000005845.2.gf.mdl', ws)
kbase_media = api.get_object('Carbon-D-Glucose', ws)
#kbase_fba = api.get_object('fva_and_ko', ws)

In [29]:
media = cobrakbase.core.KBaseBiochemMedia(kbase_media)

In [30]:
fbamodel = cobrakbase.core.KBaseFBAModel(kbase_model)

model = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(fbamodel) \
                                  .with_media(media) \
                                  .build()

In [31]:
#model = cobra.io.read_sbml_model('/Users/fliu/workspace/jupyter/data/sbml/iML1515/source.xml')
#cobra.io.write_sbml_model(model, '/Users/fliu/workspace/jupyter/data/tmp.xml', f_replace=None)

In [32]:
start_optimization = time.time()
solution = cobra.flux_analysis.pfba(model)
fva = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.1)
end_optimization = time.time()
print('optimization time(s):', end_optimization - start_optimization)

optimization time(s): 3.4134950637817383


In [33]:
solution.objective_value

135.72903761605943

In [34]:
expected_ovalue = 0.152168
print('error', (solution.objective_value - expected_ovalue)**2)

error 18381.08757488998


In [35]:
from cobrakbase.core.kbase_fba_builder import KBaseFBABuilder
b = KBaseFBABuilder.from_cobra("fva_and_ko.fba_cobrakbase", model, solution, media, ws) \
                   .with_cobra_fva_solution(fva)
fba_cobrakbase = b.build()
api.save_object(fba_cobrakbase['id'], ws, "KBaseFBA.FBA",fba_cobrakbase)

[[14,
  'fva_and_ko.fba_cobrakbase',
  'KBaseFBA.FBA-13.2',
  '2020-08-12T17:04:57+0000',
  7,
  'filipeliu',
  65434,
  'filipeliu:narrative_1592958114519',
  'a9543ed5270c1da362a02ed8abe0b33a',
  572811,
  {'Number compound variables': '183',
   'Number constraints': '0',
   'Media': '65434/5/1',
   'Number additional compounds': '0',
   'Maximized': '1',
   'Number reaction variables': '1537',
   'Number gene KO': '0',
   'Minimize reactions': '0',
   'Objective': '135.72903761605943',
   'Number compound bounds': '0',
   'Number biomass objectives': '1',
   'Combination deletions': '0',
   'Model': '65434/6/1',
   'Number reaction KO': '0',
   'Number reaction bounds': '0'}]]

In [36]:
end = time.time()
print(end - start)

6.642596960067749


In [37]:
api.ws_client.get_workspace_info({'workspace' : 'filipeliu:narrative_1592958114519'})

[65434,
 'filipeliu:narrative_1592958114519',
 'filipeliu',
 '2020-08-12T17:04:57+0000',
 14,
 'a',
 'n',
 'unlocked',
 {'cell_count': '1',
  'narrative_nice_name': 'Test CobraKBase',
  'searchtags': 'narrative',
  'is_temporary': 'false',
  'narrative': '1'}]

In [11]:
fba_cobrakbase['id']

'fva_and_ko.fba_cobrakbase'

In [12]:
assert(set(fba_cobrakbase) == set(kbase_fba))

In [13]:
f = 'FBABiomassVariables'
assert(len(kbase_fba[f]) == len(fba_cobrakbase[f]))
assert(set(kbase_fba[f][0]) == set(fba_cobrakbase[f][0]))
for k in kbase_fba[f][0]:
    try:
        assert(kbase_fba[f][0][k] == fba_cobrakbase[f][0][k])
    except AssertionError as e:
        print(k, kbase_fba[f][0][k], fba_cobrakbase[f][0][k])

value 0.152168 0.1521677489379692


In [14]:
actual = dict(map(lambda x : (x['modelreaction_ref'], x), fba_cobrakbase['FBAReactionVariables']))
expected = dict(map(lambda x : (x['modelreaction_ref'], x), kbase_fba['FBAReactionVariables']))
assert(set(actual) == set(expected))

In [15]:
fva_dict = fva.to_dict()
fva_dict.keys()

dict_keys(['minimum', 'maximum'])

In [74]:
import math
i = -8.54014e-05
print(i, round(i, 12))

-8.54014e-05 -8.54014e-05


In [60]:
fva_data = {}
for rxn_id, d in fva.iterrows():
    fva_data[rxn_id] = (d['minimum'], d['maximum'])

In [17]:
for r in expected:
    o = expected[r]
    print('class', o['class'], o['min'], o['max'], o['variableType'])
    break

class Blocked 0 0 flux


In [16]:
errors = []
for reaction_ref in expected:
    e = expected[reaction_ref]
    a = actual[reaction_ref]
    assert(set(e) == set(a))
    for k in e:
        try:
            assert(e[k] == a[k])
        except AssertionError as ex:
            errors.append([reaction_ref, k, a[k], e[k]])
            #print(reaction_ref, k, 'expected', e[k], 'actual', a[k], fva_dict['minimum'][reaction_ref.split('/')[-1]])
print(len(errors))

1740


In [24]:
assert(set(d2) == set(d1))

NameError: name 'd2' is not defined

In [197]:
print(solution['rxn00737_c0'])
b.flux_dist['rxn00737_c0']

0.0


0.0

In [30]:
import time
time.asctime()

'Tue Aug 11 23:15:32 2020'

In [35]:
import sys
sys.executable
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/Users/fliu/opt/anaconda3/bin/python
3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)


'3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

In [291]:
to_zero = ['rxn00737_c0', 'rxn00762_c0', 'rxn00935_c0', 'rxn01870_c0', 'rxn00616_c0',
           'rxn01354_c0', 'rxn09188_c0', 'rxn00173_c0', 'rxn01388_c0', 'rxn00612_c0',
           'rxn01912_c0', 'rxn01911_c0', 'rxn01682_c0', 'rxn00212_c0', 'rxn00830_c0',
           'rxn04996_c0', 'rxn08941_c0', 'rxn03954_c0', 'rxn00931_c0', 'rxn08971_c0',
           'rxn08901_c0', 'rxn08656_c0', 'rxn01302_c0', 'rxn08976_c0', 'rxn00904_c0',
           'rxn08557_c0', 'rxn00260_c0', 'rxn00541_c0', 'rxn00840_c0', 'rxn00239_c0',
           'rxn02103_c0', 'rxn01353_c0', 'rxn02102_c0'
          ]
for o in to_zero:
    r = model.reactions.get_by_id(o)
    r.upper_bound = 0
    r.lower_bound = 0

In [295]:
#difference of flux after force equal dist
0.15216774893796897 - 0.15216774893796850

4.718447854656915e-16

In [306]:
solution = model.optimize()
assert(solution.status == 'optimal')
try:
    assert(solution.objective_value == 0.15216774893796897)
except:
    print(solution.objective_value)
b = KBaseFBABuilder.fromCobra(model, solution, media, ws)
fba_cobrakbase = b.build()
d1 = dict(map(lambda o : (o['modelreaction_ref'], o), fba_cobrakbase['FBAReactionVariables']))
d2 = dict(map(lambda o : (o['modelreaction_ref'], o), kbase_fba['FBAReactionVariables']))

0.1521677489379685
bio1_biomass
DM_cpd02701_c0
DM_cpd15302_c0
DM_cpd11416_c0


In [307]:
for k in d1:
    try:
        error = (d1[k]['value'] - d2[k]['value'])**2
        assert(d1[k] == d2[k])
    except AssertionError as e:
        if error > 1e-6:
            print(k.split('/')[-1], d1[k]['value'], d2[k]['value'])

In [308]:
for k in kbase_fba:
    if type(kbase_fba[k]) == str:
        print(k, kbase_fba[k], fba_cobrakbase[k])
    elif type(kbase_fba[k]) == int:
        print(k, kbase_fba[k], fba_cobrakbase[k])
    elif type(kbase_fba[k]) == float:
        print(k, kbase_fba[k], fba_cobrakbase[k])
    elif type(kbase_fba[k]) == list:
        print(k, len(kbase_fba[k]), len(fba_cobrakbase[k]))
    else:
        print(k, type(kbase_fba[k]))

FBABiomassVariables 1 1
FBACompoundBounds 0 0
FBACompoundVariables 185 183
FBAConstraints 0 0
FBADeletionResults 0 0
FBAMetaboliteProductionResults 0 0
FBAMinimalMediaResults 0 0
FBAMinimalReactionsResults 0 0
FBAPromResults 0 0
FBAReactionBounds 0 0
FBAReactionVariables 1537 1537
FBATintleResults 0 0
MFALog  
PROMKappa 1 1
QuantitativeOptimizationSolutions 0 0
__VERSION__ 1 1
additionalCpd_refs 0 0
allReversible 0 0
biomassRemovals <class 'dict'>
biomassflux_objterms <class 'dict'>
calculateReactionKnockoutSensitivity 0 0
comboDeletions 0 0
compoundflux_objterms <class 'dict'>
decomposeReversibleDrainFlux 0 0
decomposeReversibleFlux 0 0
defaultMaxDrainFlux 0 0
defaultMaxFlux 1000 1000
defaultMinDrainFlux -1000 -1000
drainfluxUseVariables 0 0
fbamodel_ref 65434/6/1 filipeliu:narrative_1592958114519/GCF_000005845.2.gf.mdl
findMinimalMedia 0 0
fluxMinimization 1 1
fluxUseVariables 0 0
fva 0 0
gapfillingSolutions 0 0
geneKO_refs 0 0
id GCF_000005845.2.fba fba1
inputfiles <class 'dict'>
ma

In [51]:
def search(query, inlist):
    s = set(filter(lambda x : x.id == query, inlist))
    return None if len(s) == 0 else s.pop()
        
def somefunction(fba_id, model, solution, media, model_name):
    #Saving final solution as an FBA object in KBase
    fbaobj = {
        "FBABiomassVariables": [
            {
                "biomass_ref": "~/fbamodel/biomasses/id/bio1",
                "class": "unknown",
                "lowerBound": model.reactions.bio1_biomass.lower_bound,
                "max": model.reactions.bio1_biomass.upper_bound,
                "min": model.reactions.bio1_biomass.lower_bound,
                "other_max": [],
                "other_min": [],
                "other_values": [],
                "upperBound": model.reactions.bio1_biomass.upper_bound,
                "value": solution.fluxes.get("bio1_biomass"),
                "variableType": "biomassflux"
            }
        ],"FBACompoundBounds": [],
        "FBACompoundVariables": [],
        "FBAConstraints": [],
        "FBADeletionResults": [],
        "FBAMetaboliteProductionResults": [],
        "FBAMinimalMediaResults": [],
        "FBAMinimalReactionsResults": [],
        "FBAPromResults": [],
        "FBAReactionBounds": [],
        "FBAReactionVariables": [],
        "FBATintleResults": [],
        "MFALog": "",
        "PROMKappa": 1,
        "QuantitativeOptimizationSolutions": [],
        "__VERSION__": 1,
        "additionalCpd_refs": [],
        "allReversible": 0,
        "biomassRemovals": {},
        "biomassflux_objterms": {
            "bio1": 1
        },
        "calculateReactionKnockoutSensitivity": 0,
        "comboDeletions": 0,
        "compoundflux_objterms": {},
        "decomposeReversibleDrainFlux": 0,
        "decomposeReversibleFlux": 0,
        "defaultMaxDrainFlux": 0,
        "defaultMaxFlux": 1000,
        "defaultMinDrainFlux": -1000,
        "drainfluxUseVariables": 0,
        "fbamodel_ref": ws+"/"+model_name,
        "findMinimalMedia": 0,
        "fluxMinimization": 1,
        "fluxUseVariables": 0,
        "fva": 0,
        "gapfillingSolutions": [],
        "geneKO_refs": [],
        "id": fba_id,
        "inputfiles": {},
        "maximizeActiveReactions": 0,
        "maximizeObjective": 1,
        "media_list_refs": [],
        "media_ref": '{}/{}'.format(ws, media.data['name']),
        "minimizeErrorThermodynamicConstraints": 0,
        "minimize_reaction_costs": {},
        "minimize_reactions": 0,
        "noErrorThermodynamicConstraints": 0,
        "numberOfSolutions": 1,
        "objectiveConstraintFraction": 0.1,
        "objectiveValue": 0.5,
        "other_objectives": [],
        "outputfiles": {},
        "parameters": {
            "Auxotrophy metabolite list": "",
            "Beachhead metabolite list": "",
            "minimum_target_flux": "0.01",
            "save phenotype fluxes": "0",
            "suboptimal solutions": "1"
        },
        "quantitativeOptimization": 0,
        "reactionKO_refs": [],
        "reactionflux_objterms": {},
        "simpleThermoConstraints": 0,
        "thermodynamicConstraints": 0,
        "uptakeLimits": {}
    }
    series = solution.fluxes
    for varname, value in series.items():
        rxn = model.reactions.get_by_id(varname)
        
        if varname.find("EX_") >= 0:
            fbaobj["FBACompoundVariables"].append({
                "class": "Variable",
                "lowerBound": rxn.lower_bound,
                "max": rxn.upper_bound,
                "min": rxn.lower_bound,
                "upperBound": rxn.upper_bound,
                "modelcompound_ref": "~/fbamodel/modelcompounds/id/"+varname[3:],
                "other_max": [],
                "other_min": [],
                "other_values": [],
                "value": -1*value,
                "variableType": "drainflux"
            })
        elif varname.endswith("_biomass") or varname.startswith("DM_"):
            print(varname)
        else:
            fbaobj["FBAReactionVariables"].append({
                "biomass_dependencies": [],
                "class": "Variable",
                "coupled_reactions": [],
                "exp_state": "unknown",
                "expression": 0,
                "lowerBound": rxn.lower_bound,
                "max": rxn.upper_bound,
                "min": rxn.lower_bound,
                "upperBound": rxn.upper_bound,
                "modelreaction_ref": "~/fbamodel/modelreactions/id/"+varname,
                "other_max": [],
                "other_min": [],
                "other_values": [],
                "scaled_exp": 0,
                "value": value,
                "variableType": "flux"
            })

somefunction(model, solution, media, 'model1')

bio1_biomass
DM_cpd02701_c0
DM_cpd15302_c0
DM_cpd11416_c0
